In [4]:
import time
import re
from typing import Any
from specparser.amt import loader, schedules, table, tickers, strings

amt_path = "data/amt.yml"
prices_path = "data/prices.parquet"
chain_path = "data/futs.csv"
override_path = "data/overrides.csv"
start_year = 2001
end_year = 2026

amt = loader.load_amt("../data/amt.yml")
t0 = time.perf_counter()
straddle_days=schedules._find_straddle_days_legacy("../data/amt.yml", 2001, 2025, ".", True)
t1 = time.perf_counter()
current_time = t1 - t0
print(f"find_straddle_days_legacy(): {current_time:.3f}s {len(straddle_days["rows"][0]):_}")

amt = loader.load_amt("../data/amt.yml")
t0 = time.perf_counter()
straddle_days=schedules.find_straddle_days("../data/amt.yml", 2001, 2025, ".", True)
t1 = time.perf_counter()
current_time = t1 - t0
print(f"find_straddle_days(): {current_time:.3f}s {len(straddle_days["rows"][0]):_}")


print(f"rows: {len(straddle_days["rows"]):_}")
print(f"{straddle_days["orientation"]}")
#straddle_days["rows"] = list(map(list,zip(*straddle_days["rows"])))

#print(f"{len(straddle_days["rows"])}")
#table.print_table(table.table_sample(straddle_days,10))




find_straddle_days_legacy(): 0.373s 15_394_866
find_straddle_days(): 0.746s 15_394_866
rows: 3
arrow


Get schedule (with cache!)

In [2]:
table.print_table(schedules.get_schedule(str("data/amt.yml"),"LA Comdty"))

schcnt	schid	asset	ntrc	ntrv	xprc	xprv	wgt
4	1	LA Comdty	N	0	OVERRIDE		33.3
4	2	LA Comdty	N	5	OVERRIDE		33.3
4	3	LA Comdty	F	10	OVERRIDE		12.5
4	4	LA Comdty	F	15	OVERRIDE		12.5


In [3]:
from specparser.amt import loader, schedules, table, tickers, strings
table.print_table(schedules.find_schedules(str("data/amt.yml"),"LA Comdty",True))

schcnt	schid	asset	ntrc	ntrv	xprc	xprv	wgt
4	1	LA Comdty	N	0	OVERRIDE		33.3
4	2	LA Comdty	N	5	OVERRIDE		33.3
4	3	LA Comdty	F	10	OVERRIDE		12.5
4	4	LA Comdty	F	15	OVERRIDE		12.5
4	1	LA Comdty TKRZ	N	0	OVERRIDE		33.3
4	2	LA Comdty TKRZ	N	5	OVERRIDE		33.3
4	3	LA Comdty TKRZ	F	10	OVERRIDE		12.5
4	4	LA Comdty TKRZ	F	15	OVERRIDE		12.5


In [4]:
from specparser.amt import loader, schedules, table, tickers, strings
assets = table.table_column(loader.assets("data/amt.yml",live_only=True,pattern='.'),"asset")
for asset in assets[:100]:
    print(f"asset:{asset}")
    n=schedules.get_schedule_count("data/amt.yml",asset)
    table.print_table(schedules.get_schedule("data/amt.yml",asset))
    for i in range(n):
        res=tickers.get_straddle_valuation(asset,2023,1,i, "data/amt.yml","data/futs.csv","data/prices.parquet")
        print(f"asset:{asset} straddle:{i} days{len(res['rows'])}")

asset:LA Comdty
schcnt	schid	asset	ntrc	ntrv	xprc	xprv	wgt
4	1	LA Comdty	N	0	OVERRIDE		33.3
4	2	LA Comdty	N	5	OVERRIDE		33.3
4	3	LA Comdty	F	10	OVERRIDE		12.5
4	4	LA Comdty	F	15	OVERRIDE		12.5
asset:LA Comdty straddle:0 days62
asset:LA Comdty straddle:1 days62
asset:LA Comdty straddle:2 days92
asset:LA Comdty straddle:3 days92
asset:LP Comdty
schcnt	schid	asset	ntrc	ntrv	xprc	xprv	wgt
4	1	LP Comdty	N	0	OVERRIDE		33.3
4	2	LP Comdty	N	5	OVERRIDE		33.3
4	3	LP Comdty	F	10	OVERRIDE		12.5
4	4	LP Comdty	F	15	OVERRIDE		12.5
asset:LP Comdty straddle:0 days62
asset:LP Comdty straddle:1 days62
asset:LP Comdty straddle:2 days92
asset:LP Comdty straddle:3 days92
asset:LX Comdty
schcnt	schid	asset	ntrc	ntrv	xprc	xprv	wgt
4	1	LX Comdty	N	0	OVERRIDE		33.3
4	2	LX Comdty	N	5	OVERRIDE		33.3
4	3	LX Comdty	F	10	OVERRIDE		12.5
4	4	LX Comdty	F	15	OVERRIDE		12.5
asset:LX Comdty straddle:0 days62
asset:LX Comdty straddle:1 days62
asset:LX Comdty straddle:2 days92
asset:LX Comdty straddle:3 days92
asset:LA Comd

RuntimeError: Query interrupted

straddle count

In [ ]:
from specparser.amt import loader, schedules, table, tickers, strings
schedules.get_schedule_count("data/amt.yml","LA Comdty")


4

asset tickers first

In [ ]:
from specparser.amt import loader, schedules, table, tickers, strings
table.print_table(tickers.filter_tickers("LA Comdty",2023,1,0,"data/amt.yml","data/futs.csv"))

days + prices after

In [ ]:
from specparser.amt import loader, schedules, table, tickers, strings
table.print_table(tickers.get_straddle_days("LA Comdty",2023,1,0,"data/amt.yml", "data/futs.csv","data/prices.parquet"))

asset	straddle	date	vol	hedge	action	model	strike_vol	strike	expiry
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-01	26.9	2.47e+03	-	ES	-	-	-
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-02	26.4	2.53e+03	-	ES	-	-	-
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-03	none	none	-	ES	-	-	-
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-04	none	none	-	ES	-	-	-
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-05	17.1	2.51e+03	-	ES	-	-	-
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-06	27.1	2.49e+03	-	ES	-	-	-
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-07	31.2	2.47e+03	ntry	ES	31.2	2.47e+03	2023-01-04
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-08	26	2.49e+03	-	ES	31.2	2.47e+03	2023-01-04
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-09	25.8	2.46e+03	-	ES	31.2	2.47e+03	2023-01-04
LA Comdty	|2022-12|2023-01|N|0|OVERRIDE||33.3|	2022-12-10	none	none	-	ES	31.2	2.47e+03	2023-01-04
LA Comdty	|2022-12|2023-01|N|0|OVERRI

valuation finally

In [1]:
from specparser.amt import loader, schedules, table, tickers, strings
table.print_table(tickers.get_straddle_valuation("LA Comdty",2023,1,0,"data/amt.yml","data/futs.csv","data/prices.parquet"))

FileNotFoundError: [Errno 2] No such file or directory: 'data/amt.yml'

In [ ]:
import calendar
from datetime import date
from typing import List
import time


def year_month_days(
    start_year: int, start_month: int,
    end_year: int, end_month: int,
) -> List[date]:
    """
    Return all calendar days from the 1st of (start_year,start_month)
    through the last day of (end_year,end_month), inclusive.
    """
    if not (1 <= start_month <= 12 and 1 <= end_month <= 12):
        raise ValueError("month must be in 1..12")

    start = date(start_year, start_month, 1)
    end = date(end_year, end_month, calendar.monthrange(end_year, end_month)[1])
    if start > end: raise ValueError("start must be <= end")

    n_days = (end - start).days + 1
    return [start.fromordinal(start.toordinal() + i) for i in range(n_days)]

t0 = time.perf_counter()
res=year_month_days(2001,1,2001,1)
t1 = time.perf_counter()
res_time = t1 - t0
print(f"month_span_days(): {res_time:.3f}s ({len(res):,} days)")




month_span_days(): 0.000s (31 days)


In [8]:
from specparser.amt import loader, schedules, table, tickers, strings
import time
import numpy as np

#ymd = np.array(schedules.year_month_days("|2001-01|2001-12|N|0|OVERRIDE||33.3|"))
t0 = time.perf_counter()
ymd = schedules.straddle_days("|2001-01|2026-12|N|0|OVERRIDE||33.3|")
t1 = time.perf_counter()
res_time = t1 - t0
print(f"year_month_days(): {res_time:.3f}s ({len(ymd):,} days)")
print(f"days:{schedules.count_straddle_days("|2001-01|2026-12|N|0|OVERRIDE||33.3|"):,}")

year_month_days(): 0.001s (9,496 days)
days:9,496


In [9]:

import time
from specparser.amt.schedules import find_straddle_days, find_straddle_days_arrow, clear_calendar_cache

print('Performance Comparison: find_straddle_days vs find_straddle_days_arrow')
print('=' * 70)

# Warmup
clear_calendar_cache()
_ = find_straddle_days('../data/amt.yml', 2020, 2027, live_only=False)
clear_calendar_cache()
_ = find_straddle_days_arrow('../data/amt.yml', 2020, 2027, live_only=False)

# Multiple iterations
n_iter = 10
print(f'Running {n_iter} iterations each (after warmup)...')
print()

# Standard (optimized Python loop)
clear_calendar_cache()
times_std = []
for i in range(n_iter):
    t0 = time.perf_counter()
    result = find_straddle_days('../data/amt.yml', 2001, 2026, live_only=True)
    times_std.append(time.perf_counter() - t0)
row_count = len(result['rows'][0])
print(f'find_straddle_days()       : {min(times_std):.3f}s (best), {sum(times_std)/n_iter:.3f}s (avg)')
print(f'                             {row_count:,} rows, orientation: {result["orientation"]}')

# Arrow version
clear_calendar_cache()
times_arrow = []
for i in range(n_iter):
    t0 = time.perf_counter()
    result = find_straddle_days_arrow('../data/amt.yml', 2001, 2026, live_only=True)
    times_arrow.append(time.perf_counter() - t0)
row_count = len(result['rows'][0])
print(f'find_straddle_days_arrow() : {min(times_arrow):.3f}s (best), {sum(times_arrow)/n_iter:.3f}s (avg)')
print(f'                             {row_count:,} rows, orientation: {result["orientation"]}')

print()
print(f'Standard is {min(times_arrow)/min(times_std):.2f}x faster than Arrow for this dataset.')


Performance Comparison: find_straddle_days vs find_straddle_days_arrow
Running 10 iterations each (after warmup)...

find_straddle_days()       : 0.382s (best), 0.639s (avg)
                             16,010,256 rows, orientation: column
find_straddle_days_arrow() : 0.559s (best), 0.611s (avg)
                             16,010,256 rows, orientation: arrow

Standard is 1.46x faster than Arrow for this dataset.


In [2]:
import time
from specparser.amt.schedules import find_straddle_days as py, find_straddle_days_arrow as ar, find_straddle_days_numba as nb
_=nb('../data/amt.yml',2024,2024)  # JIT warmup
for n,f in [('Python',py),('Arrow',ar),('Numba',nb),('Numba-par',lambda p,s,e:nb(p,s,e,parallel=True))]:
    t=time.perf_counter()
    [f('../data/amt.yml',2001,2026) for _ in range(3)]
    print(f'{n:12s}: {(time.perf_counter()-t)/3*1000:6.1f} ms')


Python      :  646.7 ms
Arrow       :  581.9 ms
Numba       :  510.4 ms
Numba-par   :  501.3 ms


In [6]:
import pandas as pd
def show(tbl):
    """Display a table as a pandas DataFrame for pretty rendering."""
    # Convert any non-row orientation to row first
    if tbl.get("orientation") in ("column", "arrow"):
        t = table_to_rows(tbl)
    else:
        t = tbl
    return pd.DataFrame(t["rows"], columns=t["columns"])


from specparser.amt import tickers,table
import time
t=time.perf_counter()
p=tickers.get_straddle_actions('EURUSD Curncy', 2024, 1, 1, '../data/amt.yml','../data/futs.csv','../data/prices.parquet','../data/overrides.csv')
print(f'{(time.perf_counter()-t)*1000:.1f}ms, {len(p["rows"])} rows')
show(p)






33.1ms, 62 rows


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry
0,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2023-12-01,6.45,1.0884,-,ES,-,-,-
1,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2023-12-02,none,none,-,ES,-,-,-
2,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2023-12-03,none,none,-,ES,-,-,-
3,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2023-12-04,6.8,1.0836,-,ES,-,-,-
4,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2023-12-05,6.9,1.0797,-,ES,-,-,-
...,...,...,...,...,...,...,...,...,...,...
57,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2024-01-27,none,none,-,ES,-,-,-
58,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2024-01-28,none,none,-,ES,-,-,-
59,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2024-01-29,6.45,1.0833,-,ES,-,-,-
60,EURUSD Curncy,|2023-12|2024-01|N|0|BD|5|25|,2024-01-30,6.475,1.0845,-,ES,-,-,-


In [10]:
import pandas as pd
def show(tbl):
    """Display a table as a pandas DataFrame for pretty rendering."""
    # Convert any non-row orientation to row first
    if tbl.get("orientation") in ("column", "arrow"):
        t = table_to_rows(tbl)
    else:
        t = tbl
    return pd.DataFrame(t["rows"], columns=t["columns"])

from specparser.amt import tickers,table
import time
tickers.load_all_prices('../data/prices.parquet')
t=time.perf_counter()
r=tickers.get_prices('CL Comdty', 2024, 6, 1, '../data/amt.yml','../data/futs.csv','../data/prices.parquet','../data/overrides.csv')
print(f'{(time.perf_counter()-t)*1000:.1f}ms, {len(r["rows"])} rows')
show(r)

TypeError: get_prices() takes from 5 to 7 positional arguments but 8 were given

In [1]:
import pandas as pd
def show(tbl):
    """Display a table as a pandas DataFrame for pretty rendering."""
    # Convert any non-row orientation to row first
    if tbl.get("orientation") in ("column", "arrow"):
        t = table_to_rows(tbl)
    else:
        t = tbl
    return pd.DataFrame(t["rows"], columns=t["columns"])

from specparser.amt import tickers,table
import time
tickers.load_all_prices('../data/prices.parquet')
t=time.perf_counter()
r=tickers.get_straddle_valuation('CL Comdty', 2024, 6, 1, '../data/amt.yml','../data/futs.csv','../data/prices.parquet','../data/overrides.csv')
print(f'{(time.perf_counter()-t)*1000:.1f}ms, {len(r["rows"])} rows')
show(r)

314.8ms, 61 rows


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry,mv,delta,opnl,hpnl,pnl
0,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-05-01,26.36,78.44,-,ES,-,-,-,-,-,-,-,-
1,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-05-02,25.11,78.48,-,ES,-,-,-,-,-,-,-,-
2,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-05-03,24.85,77.76,-,ES,-,-,-,-,-,-,-,-
3,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-05-04,none,none,-,ES,-,-,-,-,-,-,-,-
4,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-05-05,none,none,-,ES,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-06-26,23.61,none,-,ES,-,-,-,-,-,-,-,-
57,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-06-27,23.97,none,-,ES,-,-,-,-,-,-,-,-
58,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-06-28,24.45,none,-,ES,-,-,-,-,-,-,-,-
59,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|,2024-06-29,none,none,-,ES,-,-,-,-,-,-,-,-


In [ ]:
import pandas as pd
def show(tbl):
    """Display a table as a pandas DataFrame for pretty rendering."""
    # Convert any non-row orientation to row first
    if tbl.get("orientation") in ("column", "arrow"):
        t = table_to_rows(tbl)
    else:
        t = tbl
    return pd.DataFrame(t["rows"], columns=t["columns"])

from specparser.amt import tickers,table
import time


prices_table = tickers.get_prices(
    'CL Comdty', 
    2025, 
    1, 
    0, 
    '../data/amt.yml', 
    '../data/futs.csv', 
    '../data/prices.parquet'
)

actions_table = tickers.actions(prices_table, '../data/amt.yml', '../data/overrides.csv')

show(actions_table)

,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry
0,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-01,none,none,-,ES,-,-,-
1,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-02,29.95,67.84,-,ES,-,-,-
2,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-03,28.76,69.54,-,ES,-,-,-
3,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-04,29.06,68.19,-,ES,-,-,-
4,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-05,27.66,67.96,-,ES,-,-,-
...,...,...,...,...,...,...,...,...,...,...
57,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2025-01-27,27.84,none,-,ES,-,-,-
58,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2025-01-28,27.79,none,-,ES,-,-,-
59,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2025-01-29,28.35,none,-,ES,-,-,-
60,CL Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2025-01-30,27.78,none,-,ES,-,-,-


In [1]:
from specparser.amt import tickers, schedules, table
import time

import pandas as pd
def show(tbl):
    """Display a table as a pandas DataFrame for pretty rendering."""
    # Convert any non-row orientation to row first
    if tbl.get("orientation") in ("column", "arrow"):
        t = table.table_to_rows(tbl)
    else:
        t = tbl
    return pd.DataFrame(t["rows"], columns=t["columns"])




p = schedules.find_straddle_days_numba(
    '../data/amt.yml',
    2025,
    2025,
    pattern = ".",
    live_only= True,
    parallel = False,
)

show(p)

,asset,straddle,date
0,LA Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-01
1,LA Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-02
2,LA Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-03
3,LA Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-04
4,LA Comdty,|2024-12|2025-01|N|0|OVERRIDE||33.3|,2024-12-05
...,...,...,...
615385,LOW US Equity,|2025-10|2025-12|F|15|F|3|12.5|,2025-12-27
615386,LOW US Equity,|2025-10|2025-12|F|15|F|3|12.5|,2025-12-28
615387,LOW US Equity,|2025-10|2025-12|F|15|F|3|12.5|,2025-12-29
615388,LOW US Equity,|2025-10|2025-12|F|15|F|3|12.5|,2025-12-30
